<img src='https://repository-images.githubusercontent.com/121802384/c355bb80-7d42-11e9-9e0e-4729609f9fbc' alt='WRF-Hydro Logo' width="15%"/>

# Lesson 5 - Land surface experiments

## Overview
In [Lesson 4](Lesson-4-run-options.ipynb) we experimented with different initial and boundary conditions (precipitation). 

In this lesson, we experiment with parameter adjustments to evaluate impacts on streamflow.

**NOTE: If you have not completed Lessons 1 through 4, please stop and do so now.**

### Compiling WRF-Hydro
Below are the commands to compile WRF-Hydro. We are doing a quick short-cut to edit the setEnvar.sh to make sure HYDRO_D and SPATIAL_SOIL are both active. See [Lesson 1](Lesson-1-compile.ipynb) if you have any questions on these settings.

**NOTE: You only need to do these steps if you do not already have a compiled binary. If you do already have a compiled binary, you can skip the compile steps.**

In [ ]:
%%bash
# Change to the trunk/NDHMS directory and configure for gfort
cd ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/
./configure 2

# Make a copy of the template environment variable file, setEnvars.sh
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/template/setEnvar.sh \
~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh

# Edit setEnvar.sh to turn on debug (HYDRO_D=1) and spatial soils (SPATIAL_SOIL=1)
sed -i -e 's#export HYDRO_D=0#export HYDRO_D=1#g' ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh
sed -i -e 's#export SPATIAL_SOIL=0#export SPATIAL_SOIL=1#g' ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/setEnvar.sh

# Compile the code
./compile_offline_NoahMP.sh setEnvar.sh >> compile.log 2>&1

# Check to make sure it completed successfully and confirm the correct compile-time options were set.
tail -13 ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/compile.log

## Experiment with Modified Parameters
There are a number of key parameters that impact water partitioning, storage, and movement through the WRF-Hydro model system. We have pulled many of the most important model parameters into NetCDF files to ease parameter display and manipulation, as well as to allow the parameters to vary independently in space. Key parameter files include:
* soil_properties.nc - NoahMP soil and vegetation properties (LSM grid)
* hydro2dtbl.nc - Lateral routing model soil and surface parameters (LSM grid)
* Fulldom_hires.nc - Lateral routing model high-res parameters (routing grid)
* GWBUCKPARM.nc - Groundwater baseflow bucket model parameters (groundwater basin objects)

In this lesson, we will manipulate parameters in the `soil_properties.nc` (*mfsno*, *dksat*, *bexp*, *slope*) and `Fulldom_hires.nc` (*LKSATFAC*) files.

### Step 1: Create a new template directory and and run default parameter test case
As in the first section, we will make a new simulation directory and use this as a template for creating multiple new simulation directories.

**Step 1a: Create a new template directory for the parameter experiments.**

In [ ]:
%%bash
# Make a new directory for our default baseline simulation
mkdir -p ~/wrf-hydro-training/output/lesson5/run_default_template

# Copy our model files to the simulation directory
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/*.TBL \
~/wrf-hydro-training/output/lesson5/run_default_template
cp ~/wrf-hydro-training/wrf_hydro_nwm_public/trunk/NDHMS/Run/wrf_hydro.exe \
~/wrf-hydro-training/output/lesson5/run_default_template

# Create symbolic links to large domain files
cp -as $HOME/wrf-hydro-training/example_case/FORCING \
~/wrf-hydro-training/output/lesson5/run_default_template
cp -as $HOME/wrf-hydro-training/example_case/IWAAs/RESTART \
~/wrf-hydro-training/output/lesson5/run_default_template/RESTART

# Copy the domain/parameter files so we can modify them
cp -r $HOME/wrf-hydro-training/example_case/IWAAs/DOMAIN \
~/wrf-hydro-training/output/lesson5/run_default_template/DOMAIN

# Copy namelist files
cp ~/wrf-hydro-training/example_case/IWAAs/namelist.hrldas \
~/wrf-hydro-training/output/lesson5/run_default_template
cp ~/wrf-hydro-training/example_case/IWAAs/hydro.namelist \
~/wrf-hydro-training/output/lesson5/run_default_template

**Step 1b: Change the simulation length in the namelist.hrldas file**

We want to run the model through the full spring/summer melt period, so will extend the simulation time to about 4 months (KHOUR = 2927).

In [ ]:
%%bash
sed -i -e 's/KHOUR = 1485/KHOUR = 2927/g' ~/wrf-hydro-training/output/lesson5/run_default_template/namelist.hrldas

**Step 1c: Setup and run the IWAA parameter baseline experiment**

We now make a new run directory to run the baseline simulation with IWAA parameters.

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson5/run_default_template \
~/wrf-hydro-training/output/lesson5/run_parameter_baseline

Launch the baseline run and make sure it completes successfully.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_baseline
mpirun -np 8 ./wrf_hydro.exe >> run.log 2>&1

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson5/run_parameter_baseline/diag_hydro.00000

**Step 1d: Plot the hydrograph for the baseline (default) run**

We want to take a quick look at the hydrograph for the default parameter run so we see what behavior we might want to adjust.

We will use Python and the `xarray` library to load the data and plot hydrographs. For an intro to these tools, please see [Lesson 3](Lesson-3-visualize.ipynb).

**Load the xarray python package**

In [ ]:
# Load the required libraries
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt

xr.set_options(display_style="html")
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
# Pull the channel output files into xarray object
chanobs_control = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_baseline/*CHANOBS*',
                            combine='by_coords')

# Create a dataframe of the observed flows
obs = pd.read_csv('/home/docker/wrf-hydro-training/example_case/USGS_obs.csv',dtype=str)
obs['dateTime'] = pd.to_datetime(obs['dateTime'])
obs['streamflow_cms'] = pd.to_numeric(obs['streamflow_cms'])

# Create daily aggregations
chanobs_control_daily = chanobs_control.sel(feature_id=23123539).resample(time="1D").mean()
obs_daily = obs[obs['site_no'] == '13010065'].set_index('dateTime').resample("1D").mean()

**Plot the hydrographs folr the below outlet gage:**

**Table 1. USGS Stream Gage IDs and associated WRF-Hydro feature_id indices.**

|USGS Gage ID|WRF-Hydro gage feature_id|Gage Information|
|------|:-:|---|
| 13010065 | 23123539 |https://waterdata.usgs.gov/nwis/dv/?site_no=13010065&referred_module=sw

In [ ]:
# Plot the baseline hydrograph
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for parameter experiment',fontsize=24)
chanobs_control_daily.streamflow.plot(label='Control',
                                      color='black',
                                      linestyle='--')
obs_daily.plot(ax=axes, label='Observed', color='grey')
plot_times = chanobs_control.time.data
plot_time_buffer = pd.to_timedelta(2, unit='D') 
plt.xlim([plot_times.min()-plot_time_buffer, plot_times.max()+plot_time_buffer])
plt.legend()
plt.show()

### Step 2: Modify NoahMP parameters using NCO tools

We will create a new simulation directory for our parameter manipulation experiment. 

All of the parameter file edits will be done using NCO. NCO (NetCDF Operators, http://nco.sourceforge.net/) is a set of useful utilities to manipulate NetCDF files.

We will modify:\
1.soil_properties.nc \
2.Fulldom_hires.nc

**Step 2a: Setup the parameter experiment run directory**

First make a new run directory where we can start modifying parameter files.

In [ ]:
%%bash
cp -r ~/wrf-hydro-training/output/lesson5/run_default_template \
~/wrf-hydro-training/output/lesson5/run_parameter_mods

**Step 2a: Use the NCO commands `ncks` and `ncap2` to modify the *mfsno* parameter values in the `soil_properties.nc` file.**

**MFSNO**

The Noah-MP snowpack model is a physically based, 3-layer model including liquid water storage, melt/refreeze capability, compaction, and multiple options for snow surface albedo. Snow cover fraction is estimated based on snow depth using a snow depletion curve. Snowmelt dynamics can be very sensitive to the shape of this curve. One of the snow depletion curve parameters that can be calibrated is *mfsno*, also known as the melt factor.

Here we will use the NCO command `ncap2` to modify *mfsno* based on elevation. We will append the *HGT_M* (elevation) grid from the `geo_em.d0x.nc` file using `ncks`, then use it to set our parameter values.

First, we check the current parameter values using `ncdump`. These values have been previously calibrated as part of the IWAA development, so have variable values. The default value is 2.5.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
ncdump -v mfsno soil_properties.nc | tail -n 10

We use `ncks` to append the HGT_M (elevation) variable from the `geo_em.d0x.nc` (Noah-MP static input) file into the `soil_properties.nc` file. This works since both files have the same dimensions and resolution. Then we use `ncap2` to modify the *mfsno* values based on elevation, with cells at or above 2700m getting an *mfsno* of 1.0 and cells below 2700m getting an *mfsno* of 4.0.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
# Use ncks to append HGT_M variable from geo_em.d0x.nc into soil_properties.nc.
# This version of ncks is quirky so we are doing this in 2 steps.
ncks -O -v HGT_M geo_em.d0x.nc hgt.nc
ncks -A hgt.nc soil_properties.nc

# Set mfsno based on elevation
ncap2 -O -s "where (HGT_M >= 2700) mfsno = 1.0; elsewhere mfsno = 4.0" soil_properties.nc soil_properties.nc

# Print sample values to make sure modifications worked as expected
ncdump -v mfsno soil_properties.nc | tail -n 80 | head -n 10

**Step 2b: Use the NCO command `ncap2` to modify the *dksat* and *bexp* parameter values in the `soil_properties.nc` file.**

**DKSAT and BEXP**

As with most physically-based hydrological models, the soil saturated hydraulic conductivity (dksat) controls the speed at which water moves through the subsurface. This is a sensitive parameter in the model, and while easy to measure at the point scale, dksat is tricky to estimate at the scale of kilometers. Initial values are estimated based on soil texture class, but reported ranges have large (many orders of magnitude) variability. This is a common calibration parameter, along with the related *bexp* parameter that controls how actual conductivity is scaled from saturated conductivity based on soil water content.

Here we will use the NCO command `ncap2` to increase *dksat* by a factor of 2 and *bexp* by a factor of 3.0.

First, we check the current parameter values using `ncdump`. Note that these both vary based on soil texture class, and both have been previously calibrated. The 0 values occur over waterbodies, which are not modeled in Noah-MP.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
ncdump -v dksat soil_properties.nc | tail -n 10

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
ncdump -v bexp soil_properties.nc | tail -n 10

Use `ncap2` to multiply *dksat* by a factor of 2 and *bexp* by a factor of 3. We generally use multipliers to maintain the relative patterns by texture class, but this is not required.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
ncap2 -O -s "dksat=float(dksat*2.0)" soil_properties.nc soil_properties.nc
ncdump -v dksat soil_properties.nc | tail -n 10

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
ncap2 -O -s "bexp=float(bexp*3.0)" soil_properties.nc soil_properties.nc
ncdump -v bexp soil_properties.nc | tail -n 10

**Step 2c: Use the NCO command `ncap2` to modify the *slope* parameter value in the `soil_properties.nc` file.**

**SLOPE**

There are a number of NoahMP model parameters that affect water partitioning. One important parameter that we commonly adjust is labelled *slope*. Originally estimated based on land surface topography (hence the name "slope"), the *slope* parameter is a multiplier on the bottom boundary conductivity, and therefore controls how open or closed the bottom boundary of the soil column is. Values range from 0 to 1, where 0 is a completely closed bottom boundary and 1 is completely open. Lower slope values will keep more water in the soil column, while higher values will allow more water to drain to the channel or to deeper baseflow stores, depending on the selected baseflow physics options.

Here we will use the NCO command `ncap2` to set *slope* to a value of 0.02.

First, we check the current parameter values using `ncdump`. Note that these values have been previously calibrated.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
ncdump -v slope soil_properties.nc | tail -n 10

Use `ncap2` to set *slope* to a value of 0.02 everywhere in order to restrict baseflow drainage.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
ncap2 -O -s "slope=float(slope*0+0.02)" soil_properties.nc soil_properties.nc
ncdump -v slope soil_properties.nc | tail -n 10

### Step 3: Modify terrain routing parameters using NCO tools

Now we will turn our attention to the parameters that affect lateral routing behavior.

**Step 3a: Use the NCO command `ncap2` to modify the *LKSATFAC* parameter values in the `Fulldom_hires.nc` file**

**LKSATFAC**

The `Fulldom_hires.nc` file contains a few parameters that are important for lateral flow processes. The *LKSATFAC* parameter is a multiplier on the prescribed lateral saturated hydraulic conductivity values specified in `hydro2dtbl.nc`. By default, lateral conductivity in `hydro2dtbl.nc` matches vertical conductivity specified in `soil_properties.nc`. However, in the real world we frequently see very different conductivities in the lateral direction vs. the vertical direction (due to soil stratigraphy, preferential flowpaths caused by roots and animals, etc.). *LKSATFAC* is an easy way to adjust this anisotropy, and by default it is set to 1,000.

We will use the NCO command `ncap2` to set the *LKSATFAC* value to 10.

First, we check the current parameter values. Note that these have been previously calibrated.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
ncdump -v LKSATFAC Fulldom_hires.nc | tail -n 10

Then, we modify the parameter values using `ncap2` and confirm our changes.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods/DOMAIN/
ncap2 -O -s "LKSATFAC=float(LKSATFAC*0+10)" Fulldom_hires.nc Fulldom_hires.nc
ncdump -v LKSATFAC Fulldom_hires.nc | tail -n 10

### Step 4: Run the simulation

Now we are ready to run a simulation with our new modified parameters.

In [ ]:
%%bash
cd ~/wrf-hydro-training/output/lesson5/run_parameter_mods
mpirun -np 8 ./wrf_hydro.exe >> run.log 2>&1

Check to make sure your run completed successfully.

In [ ]:
%%bash
tail -1 ~/wrf-hydro-training/output/lesson5/run_parameter_mods/diag_hydro.00000

### Results
We will now look at the differences in streamflow between our baseline run with IWAA parmeters and the simulation using our new modified parameters.

We will use Python and the `xarray` library to load the data and plot hydrographs. For an intro to these tools, please see [Lesson 3](Lesson-3-visualize.ipynb).

**Load the CHANOBS streamflow datasets**

We are going to use the CHANOBS file because it will limit the number of reaches to only those where we have specified a gage.

In [ ]:
# Pull the channel output files into xarray objects 
chanobs_param_mods = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_mods/*CHANOBS*',
                                       combine='by_coords')

# Create daily aggregations
chanobs_param_mods_daily = chanobs_param_mods.sel(feature_id=23123539).resample(time="1D").mean()

**Plot the hydrographs**

In [ ]:
# Plot the baseline and modified parameter hydrographs
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Hydrographs for parameter experiment',fontsize=24)
chanobs_control_daily.streamflow.plot(label='Control',
                                      color='black',
                                      linestyle='--')
chanobs_param_mods_daily.streamflow.plot(label='Parameter Modifications',
                                         color='red',
                                         linestyle='-')
obs_daily.streamflow_cms.plot(ax=axes, label='Observed', color='grey')
plot_times = chanobs_control_daily.time.data
plot_time_buffer = pd.to_timedelta(2, unit='D') 
plt.xlim([plot_times.min()-plot_time_buffer, plot_times.max()+plot_time_buffer])
plt.legend()
plt.show()

We see definite differences between the streamflow responses with and without parameter modifications (red solid and black dashed lines, respectively). We were able to get closer to the 3 distinct flow pulses and lower end-of-season baseflow with the new parameters.

**Load the LDASOUT land model output files**

In our simulations, we specified land surface model output (LDASOUT) files every other day (for speed). We will read those in using xarray.

In [ ]:
# Pull the LSM output files into xarray objects 
ldasout_baseline = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_baseline/*.LDASOUT*',
                            combine='by_coords')
ldasout_mods = xr.open_mfdataset('/home/docker/wrf-hydro-training/output/lesson5/run_parameter_mods/*.LDASOUT*',
                            combine='by_coords')

**Plot the mean SWE time series**

In [ ]:
# Calculate the mean snow water equivalent (SWE) across the domain
swe_baseline_avg = ldasout_baseline.SNEQV.mean(dim=('y','x'))
swe_mods_avg = ldasout_mods.SNEQV.mean(dim=('y','x')) 

# Plot the SWE time series
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Average SWE (mm)',fontsize=24)
swe_baseline_avg.plot(label='Control', color='black', linestyle='--')
swe_mods_avg.plot(label='Parameter Modifications', color='red', linestyle='-')
plt.legend()
plt.show()

**Plot the mean soil moisture time series**

In [ ]:
# Calculate the mean bottom-layer soil moisture across the domain
smois_baseline_avg = ldasout_baseline.SOIL_M.sel(soil_layers_stag = 3).mean(dim=('y','x'))
smois_mods_avg = ldasout_mods.SOIL_M.sel(soil_layers_stag = 3).mean(dim=('y','x')) 

# Plot the soil moisture time series
fig, axes = plt.subplots(ncols=1,figsize=(12, 6))
plt.suptitle('Average Soil Moisture: Bottom Layer',fontsize=24)
smois_baseline_avg.plot(label='Control', color='black', linestyle='--')
smois_mods_avg.plot(label='Parameter Modifications', color='red', linestyle='-')
plt.legend()
plt.show()

In the modified parameter simulation, we are adjusting the snowmelt timing differently for lower and higher elevations to try to capture the varying melt pulses (*mfsno*). We are closing off the 2-m soil column bottom boundary condition, which keeps more water in the soil column and less for late-season baseflow (*slope*). Our soil parameter modifications encourage faster vertical (*dksat* and *bexp*) and slower lateral (*LKSATFAC*) soil water drainage. The combined parameter modifications result in slightly lower SWE and significantly more water being stored in the soil column, as shown in the SWE and soil moisture plots above.

### Discussion
Our baseline simulation captured the initial snowmelt runoff response quite well, but was unable to capcture the full dynamics of the two later runoff pulses. We varied *mfsno*, the Noah-MP snow depletion curve "melt factor", to better separate the timing of the lower and higher elevation snow meltouts. We also decreased *slope* to reduce late season baseflow, which was too high in the intial simulation. Finally, we increased the rates of vertical drainage through the soil column (*dksat* and *bexp*) and decreased the rates of lateral flow (*LKSATFAC*), resulting in more water held in the soil column. Ideally we would evaluate these parameter choices against additional observations, such as snow and soil moisture observations, to give us a better chance of getting the right streamflow for the right reasons.

**IMPORTANT NOTE**: We abbreviated this parameter calibration exercise to fit within a short lesson. In practice, you would want to give the model time to adjust to a new parameter set by running an extended "spin-up" period before the time period you are evaluating. You would also want to expose the model to a wider range of conditions than the single season demostrated here. Model parameters calibrated to a shorter event may not transfer well to other time periods. Good practice is to calibrate your model to a wide range of conditions to minimize the impacts of forcing, gage, or model physics/parameter uncertainties.

### Other Commonly Calibrated Parameters

**SMCMAX, SMCREF, SMCWLT**

Soil water holding capacity is controlled by the porosity (*smcmax*), field capacity (*smcref*), and wilting point (*smcwlt*). These properties are specified by soil texture class, but also vary significantly in space so can be calibrated.

**RETDEPRTFAC**

The RETDEPRTFAC parameter is a multiplier on the maximum retention depth. Ponded water on the surface above this retention depth threshold can be moved around the landscape via overland flow. The default value in the code is quite small (~0.001mm, though variable by terrain slope) to allow almost all ponded water to be available for routing. However, in many regions landscape features like wetlands, small detention ponds, and heavy vegetation litter/debris can trap water on the land surface. Increasing the RETDEPRTFAC multiplier will hold more ponded water on the surface before it becomes runoff.



# Next up - Do it yourself!
This concludes Lesson 5. Spend some time creating your own parameter experiments.

The next lesson is [Lesson 6](Lesson-6-lakes.ipynb).

**IT IS BEST TO EITHER SHUTDOWN THIS LESSON OR CLOSE IT BEFORE PROCEEDING TO THE NEXT LESSON TO AVOID POSSIBLY EXCEEDING ALLOCATED MEMORY. Shutdown the lesson be either closing the browser tab for the lesson or selecting `Kernel -> Shut Down Kernel` in JupyterLab.**

© UCAR 2020